## Analysis of Sequencing Data

**This is the analysis of Tabula Muris Senis Consortium sequencing data of BL6 non-myeloid brain tissue containing the age groups 3, 18, and 24 months.**<br>
This analysis includes quality control, clustering, and annotation. Previously done clustering and cell type annotation has been preserved and was used as a guide.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
import scipy.stats as st
import sys, os
import seaborn as sns
from matplotlib import rc_context

pd.set_option('display.max_columns', None)

sc.settings.set_figure_params(
    dpi=90,
    facecolor="white",
    color_map= "viridis_r")

### Reading Data

We read the data from an .h5ad file that contains the raw count matrix with some annotations and metadata.

Information carried over is:
- age
- subtissue (brain structures)
- pre-annotated cell types (cell ontology classes)

In [ ]:
# Reading data
adata_original = sc.read_h5ad(r"C:\Users\Johann\Documents\Coding\Python\Master Thesis\Data\TMS_Brain_Nonmyeloid.h5ad")

# Making a clean copy ...
adata = ad.AnnData(adata_original.X.copy())

# ... but preserving some metdata
adata.obs["age"] = adata_original.obs["age"].values
adata.obs["cell_type_pre"] = adata_original.obs["cell_ontology_class"].values
adata.obs["subtissue"] = adata_original.obs["subtissue"].values

adata

C:\Users\Johann\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\anndata\_io\h5ad.py:267: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  return AnnData(
C:\Users\Johann\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\anndata\_io\h5ad.py:267: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  return AnnData(


AnnData object with n_obs × n_vars = 7249 × 22966
    obs: 'age', 'cell_type_pre', 'subtissue'

### Quality Control

We calculate quality control metrics.

In [22]:
# QC